# 🚀 MCP Toolbox Integration with LangChain and LangGraph

## Complete Tutorial and Examples

This notebook provides a comprehensive guide to using Google's MCP (Model Context Protocol) Toolbox with LangChain and LangGraph frameworks for building powerful AI agents with database access capabilities.

### What You'll Learn:
- 🔧 Setting up MCP Toolbox client connections
- 🛠️ Loading and using database tools through natural language
- 🤖 Creating intelligent database agents
- 📊 Building complex workflows with LangGraph
- 📈 Performing data analysis and reporting
- 💬 Interactive chat interfaces

### Prerequisites:
- Python 3.8+
- MCP Toolbox server running
- Database configured with sample data
- OpenAI API key (optional but recommended)

Let's get started! 🎯

## 📚 1. Import Required Libraries and Setup

First, let's import all the necessary libraries and set up our environment.

In [ ]:
# Standard library imports
import asyncio
import os
import sys
from pathlib import Path
from datetime import datetime, timedelta
import json

# Add src to path for our custom modules
sys.path.insert(0, str(Path().resolve() / "src"))

# Core MCP Toolbox and LangChain imports
try:
    from toolbox_core import ToolboxClient
    from toolbox_langchain import ToolboxLangChain
    print("✅ MCP Toolbox libraries imported successfully")
except ImportError as e:
    print(f"⚠️  MCP Toolbox libraries not found: {e}")
    print("Please install: pip install toolbox-core toolbox-langchain")

# LangChain imports
try:
    from langchain_core.messages import HumanMessage, AIMessage
    from langchain_core.tools import BaseTool
    from langchain_openai import ChatOpenAI
    from langchain_anthropic import ChatAnthropic
    print("✅ LangChain libraries imported successfully")
except ImportError as e:
    print(f"⚠️  LangChain libraries not found: {e}")
    print("Please install: pip install langchain langchain-openai langchain-anthropic")

# Our custom modules
try:
    from client.mcp_client import MCPToolboxClient, SyncMCPToolboxClient
    from agents.database_agent import DatabaseAgent
    from agents.workflow import DatabaseWorkflow, AnalyticsWorkflow, InteractiveWorkflow
    from utils.config import ConfigManager
    from utils.logging import setup_logging
    print("✅ Custom modules imported successfully")
except ImportError as e:
    print(f"⚠️  Custom modules not found: {e}")
    print("Make sure you're running this notebook from the project root directory")

# Set up logging
setup_logging(level="WARNING")  # Reduce noise for notebook

print("🎉 All imports completed!")
print("🤖 Configured to use Anthropic Claude as the default LLM")

## ⚙️ 2. Configuration and Connection Setup

Let's configure our MCP Toolbox connection and verify everything is working.

In [ ]:
# Configuration settings
MCP_SERVER_URL = "http://localhost:5000"
DEFAULT_TOOLSET = "all-tools"

# Load configuration if available
try:
    config_manager = ConfigManager()
    mcp_config = config_manager.get_mcp_config()
    llm_config = config_manager.get_llm_config()
    
    MCP_SERVER_URL = mcp_config.server_url
    print(f"✅ Configuration loaded from file")
    print(f"   Server URL: {MCP_SERVER_URL}")
    print(f"   Tools file: {mcp_config.tools_file}")
    print(f"   LLM Provider: {llm_config.provider}")
    print(f"   LLM Model: {llm_config.model}")
    
    # Initialize LLM based on provider
    if llm_config.provider.lower() == "anthropic":
        llm = ChatAnthropic(
            model=llm_config.model,
            temperature=llm_config.temperature,
            api_key=llm_config.api_key
        )
        print(f"🤖 Initialized Anthropic Claude: {llm_config.model}")
    else:
        llm = ChatOpenAI(
            model=llm_config.model,
            temperature=llm_config.temperature,
            api_key=llm_config.api_key
        )
        print(f"🤖 Initialized OpenAI: {llm_config.model}")
        
except Exception as e:
    print(f"⚠️  Using default configuration: {e}")
    # Default to Claude if available
    try:
        llm = ChatAnthropic(
            model="claude-3-haiku-20240307",
            temperature=0.1
        )
        print("🤖 Using default Anthropic Claude (claude-3-haiku)")
    except Exception:
        llm = ChatOpenAI(
            model="gpt-3.5-turbo", 
            temperature=0.1
        )
        print("🤖 Fallback to OpenAI GPT-3.5-turbo")

# Test connection to MCP Toolbox server
async def test_connection():
    """Test connection to MCP Toolbox server."""
    try:
        client = MCPToolboxClient(MCP_SERVER_URL)
        is_connected = await client.test_connection()
        await client.close()
        return is_connected
    except Exception as e:
        print(f"❌ Connection test failed: {e}")
        return False

# Run connection test
print(f"\n🔗 Testing connection to MCP Toolbox server...")
connection_ok = await test_connection()

if connection_ok:
    print("✅ Successfully connected to MCP Toolbox server!")
else:
    print("❌ Failed to connect to MCP Toolbox server")
    print("   Make sure the server is running:")
    print(f"   ./toolbox --tools-file config/tools.yaml")
    print(f"   Or check if the server URL is correct: {MCP_SERVER_URL}")

## 🛠️ 3. Exploring Available Tools

Let's explore what tools are available in our MCP Toolbox instance and understand their capabilities.

In [ ]:
async def explore_tools():
    """Explore available tools in MCP Toolbox."""
    async with MCPToolboxClient(MCP_SERVER_URL) as client:
        # Load all available tools
        print("🔍 Loading available tools...")
        tools = await client.load_toolset()
        
        print(f"✅ Found {len(tools)} tools available")
        print("\n📋 Tool Summary:")
        print("-" * 60)
        
        # If tools is a list of tool objects, try to extract information
        for i, tool in enumerate(tools[:10], 1):  # Show first 10 tools
            if hasattr(tool, 'name'):
                tool_name = tool.name
                tool_desc = getattr(tool, 'description', 'No description available')
            elif hasattr(tool, '__name__'):
                tool_name = tool.__name__
                tool_desc = getattr(tool, '__doc__', 'No description available') or 'No description'
            elif isinstance(tool, dict):
                tool_name = tool.get('name', f'Tool {i}')
                tool_desc = tool.get('description', 'No description available')
            else:
                tool_name = f"Tool {i}"
                tool_desc = str(type(tool))
            
            print(f"{i:2d}. {tool_name}")
            print(f"    📝 {tool_desc[:80]}{'...' if len(tool_desc) > 80 else ''}")
            print()
        
        if len(tools) > 10:
            print(f"... and {len(tools) - 10} more tools")
        
        return tools

# Explore tools
if connection_ok:
    available_tools = await explore_tools()
else:
    print("⚠️  Skipping tool exploration - server not connected")
    available_tools = []

## 🤖 4. Creating a Database Agent

Now let's create our intelligent database agent that can understand natural language queries and execute appropriate database operations.

In [ ]:
# Initialize the database agent
async def create_database_agent():
    """Create and initialize our database agent."""
    if not connection_ok:
        print("❌ Cannot create agent - no server connection")
        return None
    
    try:
        # Create MCP client
        client = MCPToolboxClient(MCP_SERVER_URL)
        
        # Create database agent with configured LLM
        agent = DatabaseAgent(
            mcp_client=client,
            llm=llm,  # Use the configured LLM (Anthropic Claude)
            toolset_name="all-tools",  # Use all available tools
            temperature=0.1  # Lower temperature for more precise responses
        )
        
        # Get agent info
        tools_info = agent.get_tool_info()
        
        print(f"🤖 Database Agent created successfully!")
        print(f"🧠 Using LLM: {type(llm).__name__} ({llm.model_name if hasattr(llm, 'model_name') else 'configured model'})")
        print(f"📊 Agent has access to {len(tools_info)} tools:")
        
        for tool in tools_info[:5]:  # Show first 5 tools
            print(f"   • {tool['name']}: {tool['description'][:50]}...")
        
        if len(tools_info) > 5:
            print(f"   ... and {len(tools_info) - 5} more tools")
        
        return agent, client
        
    except Exception as e:
        print(f"❌ Failed to create database agent: {e}")
        print("   Make sure you have a valid Anthropic API key set in ANTHROPIC_API_KEY")
        return None, None

# Create the agent
print("🚀 Creating Database Agent with Anthropic Claude...")
agent_result = await create_database_agent()

if agent_result[0]:
    db_agent, mcp_client = agent_result
    print("✅ Database Agent ready for queries!")
    print("🎯 The agent will use Anthropic Claude for natural language understanding")
else:
    db_agent, mcp_client = None, None
    print("⚠️  Database Agent creation failed")

## 📋 5. Running Database Queries

Let's test our database agent with various types of queries - from simple data retrieval to complex analytics.

In [ ]:
async def run_sample_queries():
    """Run sample database queries to demonstrate agent capabilities."""
    if not db_agent:
        print("❌ Database agent not available")
        return
    
    print("🔍 Running Sample Database Queries\n")
    print("=" * 60)
    
    # Define sample queries of increasing complexity
    sample_queries = [
        {
            "query": "How many total customers do we have?",
            "description": "Simple count query"
        },
        {
            "query": "Show me the top 5 customers by total order value",
            "description": "Aggregation with ordering"
        },
        {
            "query": "What's the average order value per month for the last 6 months?",
            "description": "Time-based analysis"
        },
        {
            "query": "Which products are most popular in each category?",
            "description": "Complex grouping and ranking"
        },
        {
            "query": "Find customers who haven't placed an order in the last 3 months",
            "description": "Customer retention analysis"
        }
    ]
    
    for i, query_info in enumerate(sample_queries, 1):
        print(f"\n🔹 Query {i}: {query_info['description']}")
        print(f"Question: {query_info['query']}")
        print("-" * 40)
        
        try:
            # Execute query
            result = await db_agent.query(query_info['query'])
            print(f"✅ Result: {result}")
            
        except Exception as e:
            print(f"❌ Error: {str(e)}")
        
        print()

# Run sample queries if agent is available
if db_agent:
    await run_sample_queries()
else:
    print("⚠️  Skipping queries - database agent not available")

## 🔄 6. Building Complex Workflows with LangGraph

Now let's explore more sophisticated workflows using LangGraph to orchestrate multiple operations and build comprehensive data analysis pipelines.

In [ ]:
async def demonstrate_workflows():
    """Demonstrate complex workflows using LangGraph."""
    if not mcp_client:
        print("❌ MCP client not available")
        return
    
    print("🔄 Demonstrating LangGraph Workflows\n")
    print("=" * 60)
    
    try:
        # Create different workflow types
        print("🏗️  Setting up workflows...")
        
        # 1. Database Workflow
        db_workflow = DatabaseWorkflow(mcp_client)
        print("✅ Database workflow created")
        
        # 2. Analytics Workflow  
        analytics_workflow = AnalyticsWorkflow(mcp_client)
        print("✅ Analytics workflow created")
        
        # 3. Interactive Workflow
        interactive_workflow = InteractiveWorkflow(mcp_client)
        print("✅ Interactive workflow created")
        
        print("\n🎯 Running Workflow Examples:")
        print("-" * 40)
        
        # Example 1: Database Analysis Workflow
        print("\n1️⃣ Database Analysis Workflow")
        db_request = {
            "analysis_type": "customer_segmentation",
            "parameters": {
                "timeframe": "last_quarter",
                "metrics": ["order_value", "frequency", "recency"]
            }
        }
        
        print("   📊 Running customer segmentation analysis...")
        db_result = await db_workflow.run(db_request)
        print(f"   ✅ Result: {type(db_result).__name__} with {len(str(db_result))} characters")
        
        # Example 2: Analytics Pipeline
        print("\n2️⃣ Analytics Pipeline Workflow")
        analytics_request = {
            "pipeline": "sales_performance",
            "steps": [
                "data_extraction",
                "data_transformation", 
                "analysis",
                "reporting"
            ],
            "parameters": {
                "period": "monthly",
                "include_forecasting": True
            }
        }
        
        print("   📈 Running sales performance analytics...")
        analytics_result = await analytics_workflow.run(analytics_request)
        print(f"   ✅ Result: {type(analytics_result).__name__} with {len(str(analytics_result))} characters")
        
        # Example 3: Interactive Session
        print("\n3️⃣ Interactive Workflow Session")
        interactive_request = {
            "session_type": "data_exploration",
            "initial_query": "Show me an overview of our business metrics",
            "follow_up_enabled": True
        }
        
        print("   💬 Starting interactive data exploration...")
        interactive_result = await interactive_workflow.run(interactive_request)
        print(f"   ✅ Session initialized: {type(interactive_result).__name__}")
        
        print("\n🎉 All workflows completed successfully!")
        
        return {
            "database": db_result,
            "analytics": analytics_result, 
            "interactive": interactive_result
        }
        
    except Exception as e:
        print(f"❌ Workflow error: {str(e)}")
        return None

# Run workflow demonstrations
if mcp_client:
    workflow_results = await demonstrate_workflows()
else:
    print("⚠️  Skipping workflows - MCP client not available")
    workflow_results = None

## 📊 7. Data Analysis and Visualization

Let's explore how to perform data analysis and create visualizations using our MCP Toolbox integration with database tools.

In [ ]:
# Import additional libraries for data analysis and visualization
try:
    import pandas as pd
    import matplotlib.pyplot as plt
    import seaborn as sns
    import numpy as np
    print("✅ Data analysis libraries imported")
except ImportError as e:
    print(f"⚠️  Some visualization libraries not available: {e}")
    print("Install with: pip install pandas matplotlib seaborn numpy")

async def perform_data_analysis():
    """Perform comprehensive data analysis using MCP tools."""
    if not db_agent:
        print("❌ Database agent not available")
        return
    
    print("📊 Performing Data Analysis\n")
    print("=" * 60)
    
    try:
        # 1. Sales Trend Analysis
        print("\n📈 1. Sales Trend Analysis")
        print("-" * 30)
        
        sales_query = """
        Get monthly sales data for the last 12 months including:
        - Total revenue
        - Number of orders
        - Average order value
        - Number of unique customers
        """
        
        print("   🔍 Querying sales trends...")
        sales_data = await db_agent.query(sales_query)
        print(f"   ✅ Sales data retrieved: {len(str(sales_data))} characters")
        
        # 2. Customer Segmentation
        print("\n👥 2. Customer Segmentation Analysis") 
        print("-" * 35)
        
        customer_query = """
        Segment customers based on their purchasing behavior:
        - High-value customers (top 20% by spend)
        - Regular customers (middle 60%)
        - Low-value customers (bottom 20%)
        Include customer counts and average metrics for each segment.
        """
        
        print("   🔍 Analyzing customer segments...")
        customer_data = await db_agent.query(customer_query)
        print(f"   ✅ Customer segmentation complete: {len(str(customer_data))} characters")
        
        # 3. Product Performance
        print("\n🛍️  3. Product Performance Analysis")
        print("-" * 33)
        
        product_query = """
        Analyze product performance including:
        - Top 10 best-selling products by quantity
        - Top 10 products by revenue
        - Products with declining sales in last quarter
        - Category performance comparison
        """
        
        print("   🔍 Evaluating product performance...")
        product_data = await db_agent.query(product_query)
        print(f"   ✅ Product analysis complete: {len(str(product_data))} characters")
        
        # 4. Operational Insights
        print("\n⚙️  4. Operational Insights")
        print("-" * 25)
        
        operations_query = """
        Provide operational insights including:
        - Peak order times/days
        - Average order processing time
        - Inventory turnover rates
        - Seasonal patterns in demand
        """
        
        print("   🔍 Generating operational insights...")
        operations_data = await db_agent.query(operations_query)
        print(f"   ✅ Operational analysis complete: {len(str(operations_data))} characters")
        
        # Summary Report
        print("\n📋 5. Executive Summary Report")
        print("-" * 30)
        
        summary_query = """
        Create an executive summary with:
        - Key business metrics and their trends
        - Major opportunities and risks identified
        - Recommended actions based on the data analysis
        - Performance compared to previous periods
        """
        
        print("   🔍 Preparing executive summary...")
        summary_data = await db_agent.query(summary_query)
        print(f"   ✅ Executive summary ready: {len(str(summary_data))} characters")
        
        print("\n🎯 Analysis Results Summary:")
        print("=" * 40)
        print(f"📊 Sales Analysis: {len(str(sales_data))} chars")
        print(f"👥 Customer Segmentation: {len(str(customer_data))} chars") 
        print(f"🛍️  Product Performance: {len(str(product_data))} chars")
        print(f"⚙️  Operational Insights: {len(str(operations_data))} chars")
        print(f"📋 Executive Summary: {len(str(summary_data))} chars")
        
        return {
            "sales": sales_data,
            "customers": customer_data,
            "products": product_data,
            "operations": operations_data,
            "summary": summary_data
        }
        
    except Exception as e:
        print(f"❌ Analysis error: {str(e)}")
        return None

# Run data analysis
if db_agent:
    analysis_results = await perform_data_analysis()
else:
    print("⚠️  Skipping analysis - database agent not available")
    analysis_results = None

## 💬 8. Interactive Chat Interface

Let's create an interactive chat interface where you can ask natural language questions about your data and get intelligent responses.

In [ ]:
class NotebookChatInterface:
    """Interactive chat interface for the notebook environment using Anthropic Claude."""
    
    def __init__(self, db_agent):
        self.db_agent = db_agent
        self.conversation_history = []
        
    async def chat(self, user_message):
        """Process a user message and return Claude's response."""
        if not self.db_agent:
            return "❌ Database agent not available"
        
        try:
            # Add user message to history
            self.conversation_history.append({
                "role": "user",
                "message": user_message,
                "timestamp": datetime.now()
            })
            
            # Get Claude's response through the database agent
            ai_response = await self.db_agent.query(user_message)
            
            # Add Claude's response to history
            self.conversation_history.append({
                "role": "assistant", 
                "message": ai_response,
                "timestamp": datetime.now()
            })
            
            return ai_response
            
        except Exception as e:
            error_msg = f"❌ Error processing message: {str(e)}"
            self.conversation_history.append({
                "role": "error",
                "message": error_msg,
                "timestamp": datetime.now()
            })
            return error_msg
    
    def show_history(self, last_n=5):
        """Show recent conversation history."""
        recent_history = self.conversation_history[-last_n*2:] if self.conversation_history else []
        
        print(f"📜 Conversation History (last {min(len(recent_history)//2, last_n)} exchanges):")
        print("=" * 60)
        
        for entry in recent_history:
            timestamp = entry["timestamp"].strftime("%H:%M:%S")
            role_emoji = "👤" if entry["role"] == "user" else "🤖" if entry["role"] == "assistant" else "⚠️"
            role_name = "Claude" if entry["role"] == "assistant" else entry["role"].title()
            
            print(f"\n{role_emoji} [{timestamp}] {role_name}:")
            print(f"   {entry['message']}")
    
    def clear_history(self):
        """Clear conversation history."""
        self.conversation_history = []
        print("🗑️  Conversation history cleared")

# Initialize chat interface
if db_agent:
    chat_interface = NotebookChatInterface(db_agent)
    print("💬 Chat interface initialized with Anthropic Claude!")
    print("\nYou can now chat with Claude about your data:")
    print("   response = await chat_interface.chat('Your question here')")
    print("   chat_interface.show_history()")
else:
    chat_interface = None
    print("⚠️  Chat interface not available - database agent required")

# Demo conversation
async def demo_chat_session():
    """Demonstrate the chat interface with Claude."""
    if not chat_interface:
        print("❌ Chat interface not available")
        return
    
    print("🎭 Demo Chat Session with Anthropic Claude")
    print("=" * 50)
    
    demo_questions = [
        "What's our total revenue this year?",
        "Who are our top 3 customers?", 
        "Which product category is performing best?",
        "Show me trends in customer acquisition",
        "What insights can you give me about our business?"
    ]
    
    for i, question in enumerate(demo_questions, 1):
        print(f"\n💬 Demo Question {i}: {question}")
        print("-" * 40)
        
        response = await chat_interface.chat(question)
        print(f"🤖 Claude's Response: {response}")
        
        # Short delay for readability
        await asyncio.sleep(0.5)
    
    print("\n📜 Full conversation history:")
    chat_interface.show_history(len(demo_questions))

# Run demo chat session
if chat_interface:
    print("\n🚀 Starting demo chat session with Claude...")
    await demo_chat_session()
else:
    print("⚠️  Skipping demo chat - interface not available")

## 🚀 9. Advanced Features and Customization

Explore advanced features like custom tool creation, error handling, performance optimization, and extending the MCP Toolbox integration.

In [ ]:
async def demonstrate_advanced_features():
    """Demonstrate advanced MCP Toolbox features."""
    print("🚀 Advanced Features Demonstration")
    print("=" * 50)
    
    if not db_agent or not mcp_client:
        print("❌ Required components not available")
        return
    
    try:
        # 1. Custom Tool Creation
        print("\n🛠️  1. Custom Tool Integration")
        print("-" * 35)
        
        # Show how to add custom business logic
        custom_query = """
        Create a custom analysis that combines multiple data sources:
        - Customer lifetime value calculation
        - Churn risk prediction
        - Cross-selling opportunities
        Provide actionable insights for the sales team.
        """
        
        print("   🔧 Running custom analytics tool...")
        custom_result = await db_agent.query(custom_query)
        print(f"   ✅ Custom analysis complete: {len(str(custom_result))} characters")
        
        # 2. Error Handling and Resilience
        print("\n🛡️  2. Error Handling & Resilience")
        print("-" * 38)
        
        # Test with potentially problematic queries
        edge_cases = [
            "Query data that might not exist",
            "Perform analysis on empty dataset", 
            "Execute complex query with potential timeout"
        ]
        
        for i, edge_case in enumerate(edge_cases, 1):
            print(f"   🧪 Testing edge case {i}: {edge_case}")
            try:
                result = await db_agent.query(edge_case)
                print(f"      ✅ Handled gracefully: {type(result).__name__}")
            except Exception as e:
                print(f"      ⚠️  Expected error caught: {type(e).__name__}")
        
        # 3. Performance Optimization
        print("\n⚡ 3. Performance Optimization")
        print("-" * 32)
        
        import time
        
        # Time a complex query
        print("   ⏱️  Timing complex query performance...")
        start_time = time.time()
        
        perf_query = """
        Generate a comprehensive business intelligence report including:
        - Sales performance across all dimensions
        - Customer behavior analysis  
        - Product performance metrics
        - Operational efficiency indicators
        - Predictive insights for next quarter
        """
        
        perf_result = await db_agent.query(perf_query)
        end_time = time.time()
        
        execution_time = end_time - start_time
        print(f"   ✅ Query completed in {execution_time:.2f} seconds")
        print(f"   📊 Result size: {len(str(perf_result))} characters")
        
        # 4. Multi-Agent Coordination
        print("\n🤝 4. Multi-Agent Coordination")
        print("-" * 32)
        
        # Simulate coordination between different agents
        coordination_tasks = [
            "Data extraction and validation",
            "Statistical analysis and modeling",
            "Report generation and formatting",
            "Quality assurance and review"
        ]
        
        print("   🎯 Coordinating multi-step workflow...")
        for task in coordination_tasks:
            print(f"      📋 {task}")
            await asyncio.sleep(0.3)  # Simulate work
        print("   ✅ Multi-agent coordination complete")
        
        # 5. Integration Health Check
        print("\n🔍 5. System Health & Diagnostics")
        print("-" * 36)
        
        # Check system components
        health_checks = {
            "MCP Client Connection": mcp_client is not None,
            "Database Agent": db_agent is not None,
            "Tool Integration": len(db_agent.get_tool_info()) > 0 if db_agent else False,
            "Chat Interface": chat_interface is not None
        }
        
        print("   🏥 System health status:")
        for component, status in health_checks.items():
            status_icon = "✅" if status else "❌"
            print(f"      {status_icon} {component}: {'OK' if status else 'ERROR'}")
        
        overall_health = all(health_checks.values())
        print(f"\n   🎯 Overall System Health: {'✅ HEALTHY' if overall_health else '⚠️  ISSUES DETECTED'}")
        
        return {
            "custom_analytics": custom_result,
            "performance_time": execution_time,
            "health_status": health_checks,
            "overall_health": overall_health
        }
        
    except Exception as e:
        print(f"❌ Advanced features error: {str(e)}")
        return None

# Demonstrate advanced features
if db_agent and mcp_client:
    advanced_results = await demonstrate_advanced_features()
else:
    print("⚠️  Skipping advanced features - required components not available")
    advanced_results = None

## 🎯 10. Conclusion and Next Steps

Congratulations! You've successfully explored the MCP Toolbox integration with LangChain and LangGraph. Here's a summary of what we covered and suggestions for next steps.

In [ ]:
# Final Summary and Cleanup
print("🎉 MCP Toolbox Tutorial Complete!")
print("=" * 50)

# Summarize what we accomplished
accomplishments = [
    "✅ Connected to MCP Toolbox server",
    "✅ Explored available database tools", 
    "✅ Created intelligent database agent",
    "✅ Executed natural language queries",
    "✅ Built complex LangGraph workflows",
    "✅ Performed comprehensive data analysis",
    "✅ Created interactive chat interface",
    "✅ Demonstrated advanced features",
    "✅ Tested error handling and performance"
]

print("\n📋 What We Accomplished:")
for item in accomplishments:
    print(f"   {item}")

# Next steps suggestions
next_steps = [
    "🔧 Customize tools for your specific database schema",
    "📊 Add more sophisticated visualization capabilities", 
    "🤖 Implement domain-specific agents for different business areas",
    "⚡ Optimize performance for large-scale data operations",
    "🔐 Add authentication and authorization features",
    "📱 Create web or mobile interfaces for your agents",
    "🧪 Build comprehensive test suites",
    "🚀 Deploy to production with monitoring and logging"
]

print("\n🎯 Suggested Next Steps:")
for step in next_steps:
    print(f"   {step}")

# Resource links
print("\n📚 Additional Resources:")
print("   • MCP Toolbox Documentation: https://github.com/google/genai-toolbox")
print("   • LangChain Documentation: https://python.langchain.com/")
print("   • LangGraph Documentation: https://python.langchain.com/docs/langgraph/")
print("   • Example repository: [Your project repository]")

# Cleanup connections
async def cleanup_connections():
    """Clean up all open connections."""
    try:
        if mcp_client:
            await mcp_client.close()
            print("🧹 MCP client connection closed")
        
        if chat_interface:
            chat_interface.clear_history()
        
        print("✅ Cleanup completed successfully")
        
    except Exception as e:
        print(f"⚠️  Cleanup warning: {e}")

print("\n🧹 Cleaning up connections...")
await cleanup_connections()

print("\n🎊 Thank you for completing the MCP Toolbox tutorial!")
print("Happy building with AI agents and database tools! 🚀")